In [8]:
import sys
import os
import urllib.request
import tarfile
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import pickle

def _print_download_progress(count, block_size, total_size):
    pct_complete = float(count * block_size) / total_size
    msg = "\r- Download progress: {0:.1%}".format(pct_complete)
    sys.stdout.write(msg)
    sys.stdout.flush()

def maybe_download_and_extract(url, download_dir):
    filename = url.split('/')[-1]
    file_path = os.path.join(download_dir, filename)

    if not os.path.exists(file_path):
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)

        file_path, _ = urllib.request.urlretrieve(url=url,
                                                  filename=file_path,
                                                  reporthook=_print_download_progress)

        print()
        print("Download finished. Extracting files.")

        if file_path.endswith(".zip"):
            zipfile.ZipFile(file=file_path, mode="r").extractall(download_dir)
        elif file_path.endswith((".tar.gz", ".tgz")):
            tarfile.open(name=file_path, mode="r:gz").extractall(download_dir)

        print("Done.")
    else:
        print("Data has apparently already been downloaded and unpacked.")

def load_pickle(f):
    return pickle.load(f, encoding='latin1')

def load_CIFAR_batch(filename):
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
        Y = np.array(Y)
        return X, Y

def load_CIFAR10(ROOT):
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

class KNearestNeighbor(object):
    def __init__(self):
        pass

    def train(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X, k=1, num_loops=0):
        if num_loops == 0:
            dists = self.compute_distances(X)
        else:
            raise ValueError('Invalid value %d for num_loops' % num_loops)
        return self.predict_labels(dists, k=k)

    def compute_distances(self, X, distance_metric='euclidean'):
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train))
        if distance_metric == 'euclidean':
            for i in range(num_test):
                dists[i, :] = np.sqrt(np.sum((self.X_train - X[i, :])**2, axis=1))
        elif distance_metric == 'manhattan':
            for i in range(num_test):
                dists[i, :] = np.sum(np.abs(self.X_train - X[i, :]), axis=1)
        else:
            raise ValueError('Invalid distance_metric.')
        return dists

    def predict_labels(self, dists, k=1):
        num_test = dists.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):
            closest_y = []
            sorted_indices = np.argsort(dists[i, :])
            closest_y = self.y_train[sorted_indices[:k]]
            counts = np.bincount(closest_y)
            y_pred[i] = np.argmax(counts)
        return y_pred

def visualize_data(X_train, y_train):
    classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    num_classes = len(classes)
    samples_per_class = 7
    for y, cls in enumerate(classes):
        idxs = np.flatnonzero(y_train == y)
        idxs = np.random.choice(idxs, samples_per_class, replace=False)
        for i, idx in enumerate(idxs):
            plt_idx = i * num_classes + y + 1
            plt.subplot(samples_per_class, num_classes, plt_idx)
            plt.imshow(X_train[idx].astype('uint8'))
            plt.axis('off')
            if i == 0:
                plt.title(cls)
    plt.show()

if __name__ == "__main__":
    # Download CIFAR10 data and store it in the current directory if you have not done it.
    url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    download_dir = "./data"
    maybe_download_and_extract(url, download_dir)

    # Load training and testing data from CIFAR10 dataset
    cifar10_dir = './data/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # Checking the size of the training and testing data
    print('Training data shape:', X_train.shape)
    print('Training labels shape:', y_train.shape)
    print('Test data shape:', X_test.shape)
    print('Test labels shape:', y_test.shape)

    # Visualize the data if you want
    # visualize_data(X_train, y_train)

    # Memory error prevention by subsampling data. We sample 1000 training examples and 100 test examples.
    num_training = 1000
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]

    num_test = 100
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Reshape data and place into rows. Flatten the training and test data so each row
    # consists of all pixels of an example
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    print(X_train.shape, X_test.shape) # X_train should be (10000, 3072) and X_test should be (1000, 3072)

    # Performing KNN
    classifier = KNearestNeighbor()

    # Use the KNearestNeighbour classifier to do as follows:
    # 1) Initialize classifier with training data
    # 2) Use classifier to compute distances from each test example in X_test to every training example
    # 3) Use classifier to predict labels of each test example in X_test using k=5
    # Initialize classifier with training data
    classifier.train(X_train, y_train)
    dists = classifier.compute_distances(X_test)
    k = 5
    y_test_pred = classifier.predict_labels(dists, k)

    # Calculate accuracy
    num_correct = np.sum(y_test_pred == y_test)  # number of test examples correctly predicted
    accuracy = float(num_correct) / num_test
    print('Got %d / %d correct with k=%d => accuracy: %f' % (num_correct, num_test, k, accuracy))


Data has apparently already been downloaded and unpacked.
Training data shape: (50000, 32, 32, 3)
Training labels shape: (50000,)
Test data shape: (10000, 32, 32, 3)
Test labels shape: (10000,)
(1000, 3072) (100, 3072)
Got 24 / 100 correct with k=5 => accuracy: 0.240000
